# Bank Transaction Monitoring & Data Pipeline Simulation project using Spark SQL
## Objective:
To build a scalable data pipeline that ingests raw bank transactions, transforms them using Spark SQL, and analyzes them to detect potential fraudulent behavior. The goal is to demonstrate end-to-end ETL and analytics on large-scale financial data using Spark SQL in Databricks.
## Tools & Technologies Used:
- Databricks (Runtime 12.2 LTS)
- Apache Spark 3.3.2
- Spark SQL
- PySpark (for ETL)
- CSV File (Simulated banking transactions)

## Dataset Description:
The dataset contains fields like:

step: Time step of the transaction

type: Transaction type (PAYMENT, TRANSFER, etc.)

amount: Amount transacted

nameOrig, nameDest: Sender & receiver IDs

oldbalanceOrg, newbalanceOrig: Sender's balance before & after

oldbalanceDest, newbalanceDest: Receiver's balance before & after

isFraud: Fraud indicator

isFlaggedFraud: Flagged as suspicious by system

## Business Context:
Banks and financial institutions process millions of transactions daily. Identifying suspicious patterns in real-time is critical for fraud prevention. This project simulates a data engineering pipeline where raw banking transactions are ingested, cleaned, analyzed, and used to highlight fraud trends.
## Why Spark SQL and Databricks?
Scalability: Handles large datasets efficiently with distributed computing.

Speed: In-memory processing provides faster transformations than traditional SQL engines.

Ease of Use: Spark SQL syntax is similar to SQL, making it beginner-friendly for data analysis and transformation.

Integrated Workspace: Databricks allows you to develop notebooks, run Spark jobs, and manage clusters—all in one place.

Industry-Relevant: Spark + Databricks is widely used in top tech and finance companies for real-time fraud detection, stream processing, and data engineering.
## Three-Layer Architecture:
Ingestion Layer: Collects raw data from various sources and loads it into the data pipeline.

Processing Layer: Cleans, transforms, and enriches the raw data, preparing it for analysis.

Service Layer: Provides processed and aggregated data in an accessible format for consumption by applications or users.

### 1.Ingestion Layer (Raw Data Inflow)
#### Goal: Bring raw transactional data into Databricks for processing.

In [0]:
#Upload raw CSV  to Databricks FileStore
raw_df = spark.read.csv("dbfs:/FileStore/shared_uploads/bh.shreya99@gmail.com/banking_transaction_data.csv", header=True, inferSchema=True)
raw_df.show(5)
#Read the data using Spark



+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|   181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1| PAYMENT|11668.14|C2048537720|      41554.0|      29885.86|M1230701703|      

In [0]:
# Create Temporary View:
raw_df.createOrReplaceTempView("transactions")


### Interpretation:

This command creates a temporary SQL table named transactions from the DataFrame df.
This step is needed to run SQL queries on the DataFrame using Spark SQL in Databricks.

## 2. Processing Layer (Data Cleaning, Transformation, Analysis)

### Step 1: Data Cleaning (Null & Duplicate Check)

In [0]:
# Null Check
from pyspark.sql.functions import col, when, count
raw_df.select([count(when(col(c).isNull(), c)).alias(c) for c in raw_df.columns]).show()

# Duplicate Check
raw_df.groupBy(raw_df.columns).count().filter("count > 1").show()


+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+
|step|type|amount|nameOrig|oldbalanceOrg|newbalanceOrig|nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+
|   0|   0|     0|       0|            0|             0|       0|             0|             0|      0|             0|
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+

+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+-----+
|step|type|amount|nameOrig|oldbalanceOrg|newbalanceOrig|nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|count|
+----+----+------+--------+-------------+--------------+--------+--------------+--------------+-------+--------------+-----+
+----+----+------+--------+--

### Interpretation:

This dataset is complete — there are no missing or blank entries in any of the columns like step, amount, nameOrig, isFraud, etc.

Again, the dataset has no exact duplicate rows. Every row is unique based on all columns.

### Step 2: Understanding Schema

In [0]:
raw_df.printSchema()


root
 |-- step: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: integer (nullable = true)
 |-- isFlaggedFraud: integer (nullable = true)



## Step3: Total Transaction:Understand overall volume of transaction records.

In [0]:
spark.sql("SELECT COUNT(*) AS total_transactions FROM transactions").show()

+------------------+
|total_transactions|
+------------------+
|           1048575|
+------------------+



### Interpretation:
This table shows that there are a total of 1,048,575 transactions, indicating a large dataset suitable for robust analysis.

## Step 4: Transaction types summary

In [0]:
spark.sql("""
    SELECT type, COUNT(*) AS count
    FROM transactions
    GROUP BY type
    ORDER BY count DESC
""").show()



+--------+------+
|    type| count|
+--------+------+
|CASH_OUT|373641|
| PAYMENT|353873|
| CASH_IN|227130|
|TRANSFER| 86753|
|   DEBIT|  7178|
+--------+------+



### Interpretation:
This step will analyze how many transactions are there for each type (like PAYMENT, TRANSFER, etc.).

This table displays the counts of different transaction types, revealing that CASH_OUT and PAYMENT are the most frequent, while DEBIT transactions are significantly less common.
CASH_OUT and PAYMENT are the most frequent transaction types, suggesting they play a major role in overall transaction volume and may require closer monitoring.

## Step 5: Total Amount Transacted per Type: Understand financial impact of each transaction type

In [0]:
# Execute the Spark SQL query
result = spark.sql("""
    SELECT type, 
           ROUND(SUM(amount), 2) AS total_amount 
    FROM transactions 
    GROUP BY type 
    ORDER BY total_amount DESC
""")

# Show the result
result.show()





+--------+-----------------+
|    type|     total_amount|
+--------+-----------------+
|CASH_OUT|6.876473738324E10|
|TRANSFER|5.503643567413E10|
| CASH_IN|3.859475828889E10|
| PAYMENT|  3.93609652216E9|
|   DEBIT|    4.219599495E7|
+--------+-----------------+



%md
### Interpretation:
This table displays the total aggregated amounts for each transaction type, showing that CASH_OUT and TRANSFER involve significantly higher sums of money compared to other transaction types like DEBIT and PAYMENT.

CASH_OUT and TRANSFER are the dominant transaction types, indicating potential areas to focus fraud detection and resource allocation.

## Step 6: Fraud Detection Summary: Identify which transaction types are most vulnerable to fraud.

In [0]:
spark.sql("""
    SELECT type, 
           COUNT(*) AS total, 
           SUM(isFraud) AS frauds 
    FROM transactions 
    GROUP BY type
""").show()


+--------+------+------+
|    type| total|frauds|
+--------+------+------+
|TRANSFER| 86753|   564|
| CASH_IN|227130|     0|
|CASH_OUT|373641|   578|
| PAYMENT|353873|     0|
|   DEBIT|  7178|     0|
+--------+------+------+



###  Interpretation:
This table shows the total count and the number of fraudulent transactions for each transaction type, indicating that only TRANSFER and CASH_OUT types contain fraudulent activities.

## Step 7:  Flagged Frauds vs Actual Frauds: Compare system's flagged frauds vs real frauds

In [0]:
spark.sql("""
    SELECT
      SUM(CASE WHEN isFraud = 1 THEN 1 ELSE 0 END) AS actual_frauds,
      SUM(CASE WHEN isFlaggedFraud = 1 THEN 1 ELSE 0 END) AS flagged_frauds
    FROM transactions
""").show()


+-------------+--------------+
|actual_frauds|flagged_frauds|
+-------------+--------------+
|         1142|             0|
+-------------+--------------+



### Interpretation:
The table indicates a significant problem: there are 1,142 instances of actual fraud, but the existing system, which should flag fraudulent transactions, has identified zero of them. This means the current fraud detection mechanism is completely ineffective in catching these fraudulent activities.

## Step 8: High-Value Fraudulent Transactions: Investigate top high-risk transactions

In [0]:
spark.sql("""
    SELECT type, amount, nameOrig, nameDest, step
    FROM transactions
    WHERE isFraud = 1 AND amount > 200000
    ORDER BY amount DESC
    LIMIT 10
""").show()


+--------+------+-----------+-----------+----+
|    type|amount|   nameOrig|   nameDest|step|
+--------+------+-----------+-----------+----+
|TRANSFER| 1.0E7| C792651637| C397396936|  84|
|CASH_OUT| 1.0E7| C351297720| C766681183|   4|
|CASH_OUT| 1.0E7|  C29118015|C1379703840|  33|
|TRANSFER| 1.0E7|   C7162498| C945327594|   4|
|TRANSFER| 1.0E7| C416779475| C380259496|  19|
|TRANSFER| 1.0E7|C1439740840| C875288652|  33|
|CASH_OUT| 1.0E7|C1079335762| C615227407|  82|
|CASH_OUT| 1.0E7|C2050703310|C1622860679|  19|
|TRANSFER| 1.0E7|  C53057884| C588547519|  72|
|CASH_OUT| 1.0E7|C1438388258|C1089455271|  72|
+--------+------+-----------+-----------+----+



### Interpretation:
This query highlights the top 10 high-value fraud transactions, helping to detect major fraud attempts based on amount.

High-value fraudulent transactions predominantly occur via TRANSFER and CASH_OUT methods, indicating these types are prime targets for large-scale fraud in the system.

## Step 9: Account Balance Anomalies in Fraud Cases: Spot cases where funds were not credited to destination — suspicious behavior.

In [0]:
insightful_fraud = spark.sql("""
SELECT 
    type,
    COUNT(*) AS total_fraud_cases,
    ROUND(AVG(amount), 2) AS avg_amount,
    MAX(amount) AS max_amount,
    MIN(amount) AS min_amount
FROM transactions
WHERE isFraud = 1 AND oldbalanceDest = 0 AND newbalanceDest = 0
GROUP BY type
ORDER BY total_fraud_cases DESC
""")

insightful_fraud.show()




+--------+-----------------+----------+----------+----------+
|    type|total_fraud_cases|avg_amount|max_amount|min_amount|
+--------+-----------------+----------+----------+----------+
|TRANSFER|              542|1229479.35|     1.0E7|     119.0|
|CASH_OUT|                1|   82806.2|   82806.2|   82806.2|
+--------+-----------------+----------+----------+----------+




### Interpretation:
This analysis shows the frequency and scale of fraudulent transactions where the destination account remained uncredited (possibly indicating fake accounts or failed transfers). Most of these were high-value TRANSFER and CASH_OUT transactions.

This table reveals that while TRANSFER transactions account for almost all fraud cases (542 vs. 1), they also exhibit a wide range of fraudulent amounts, including extremely high values (1.0×10 
7
 ), whereas the single CASH_OUT fraud case has a fixed amount.

## 3. Serving Layer (Insights Sharing / Dashboard / Reports)

### 1. Transaction Type Summary

In [0]:

raw_df.createOrReplaceTempView("curated_transactions")
df1 = spark.sql("SELECT type, COUNT(*) AS count FROM curated_transactions GROUP BY type")
display(df1)  


type,count
TRANSFER,86753
CASH_IN,227130
CASH_OUT,373641
PAYMENT,353873
DEBIT,7178


Databricks visualization. Run in Databricks to view.

### 2. Total Amount Transacted per Type

In [0]:
df3 = spark.sql("SELECT type, ROUND(SUM(amount), 2) AS total_amount FROM curated_transactions GROUP BY type")
display(df3)  


type,total_amount
TRANSFER,5.503643567413E10
CASH_IN,3.859475828889E10
CASH_OUT,6.876473738324E10
PAYMENT,3.93609652216E9
DEBIT,4.219599495E7


Databricks visualization. Run in Databricks to view.

### 3.Fraud Detection Summary


In [0]:
df4 = spark.sql("""
SELECT type, 
       COUNT(*) AS total, 
       SUM(CASE WHEN isFraud=1 THEN 1 ELSE 0 END) AS total_frauds
FROM curated_transactions
GROUP BY type
""")
display(df4)  


type,total,total_frauds
TRANSFER,86753,564
CASH_IN,227130,0
CASH_OUT,373641,578
PAYMENT,353873,0
DEBIT,7178,0


Databricks visualization. Run in Databricks to view.

### Final Conclusion
This project involved analyzing a large-scale financial transaction dataset using Spark SQL and PySpark on Databricks to uncover insights related to fraud detection. The analysis revealed key patterns:

CASH_OUT and TRANSFER transactions dominate both in volume and value, making them critical areas for monitoring.

Fraudulent activities are exclusively found in TRANSFER and CASH_OUT transactions, pointing to specific vulnerabilities in these types.

The current fraud detection system flagged zero fraudulent transactions, while 1,142 actual frauds were identified, indicating a complete failure of the fraud flagging mechanism.

High-value frauds are concentrated in TRANSFER transactions, including some in the range of 10 million units, which is alarming.

Account balance anomalies were found where funds were debited from the origin but not credited to the destination, especially in fraudulent cases — signaling suspicious manipulations.

This analysis demonstrates the power of Spark-based data engineering pipelines and how they can be used to uncover weaknesses in fraud detection systems. The project simulates an end-to-end data pipeline from raw to cleansed to curated layers, followed by business insights visualization — preparing the foundation for deploying fraud detection ML models in future work.